In [1]:
%reload_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import numpy as np
from scipy import stats
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tueplots import bundles
from pingouin import partial_corr

bundles.beamer_moml()
# Increase the resolution of all the plots below
plt.rcParams.update({"figure.dpi": 200,"figure.facecolor":"w","figure.figsize": (10,8)})

from src.config import dir_rawdata, dir_prep, dir_plots


In [2]:
dat_pupil = pd.read_csv(Path.joinpath(dir_prep,"all_trials_pupil.csv"), index_col=False)
dat_gss = pd.read_csv(Path.joinpath(dir_prep,"all_trials_gss.csv"), index_col=False)
dat_clin = pd.read_excel(Path.joinpath(dir_rawdata,"behav_data.xlsx"), index_col=False)


['ID', 'Trial n', 'Feedback type', 'Feedback angle', 'Pupil size', 'Percentage bad pupil samples', 'Group']


In [3]:
dat_clin.head(15)

,ID,Alter,Geschlecht,Krankheitsdauer,BDI-Score,Schahmann semantische Wortflüssigkeit,Schahmann phonematische Wortflüssigkeit,Schahmann Kategorie-Wechsel,Schahmann Zahlen-spanne vorwärts,Schahmann Zahlen-spanne rückwärts,...,Tetras Tremor OE kinet. Tremor LI,Tetras Tremor UE,Tetras Archimedes RE,Tetras Archimedes LI,Tetras Handschrift,Tetras Punktan-näherung RE,Tetras Punktan-näherung LI,Tetras Tremor im Stehen,Tetras Gesamtscore Aufgaben-skala,Gesamtscore Tetras
0,p114,35.0,m,NaN,4.0,26.0,10.0,12.0,6.0,4.0,...,1.5,2.0,1.0,2.0,1.0,1.0,1.5,0.0,17.0,33.0
1,p115,46.0,w,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,1.5,1.0,1.0,1.0,1.0,1.5,2.0,0.0,17.5,35.5
2,p116,23.0,m,NaN,5.0,26.0,13.0,15.0,5.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,p117,81.0,m,NaN,13.0,24.0,7.0,9.0,4.0,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,p118,51.0,m,NaN,11.0,26.0,17.0,15.0,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,p119,72.0,m,NaN,2.0,14.0,8.0,9.0,4.0,3.0,...,1.5,0.0,2.0,2.0,3.0,1.5,1.5,0.0,22.5,38.5
6,p120,81.0,m,NaN,2.0,15.0,7.0,5.0,6.0,4.0,...,1.5,1.0,2.0,3.0,2.0,1.5,1.5,0.0,18.5,39.5
7,p121,68.0,m,NaN,6.0,23.0,12.0,12.0,7.0,5.0,...,3.0,0.0,4.0,4.0,3.0,2.0,2.0,0.0,29.5,58.5
8,p214,NaN,NaN,NaN,NaN,24.0,15.0,13.0,6.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,p109,52.0,m,NaN,4.0,26.0,11.0,15.0,7.0,5.0,...,1.0,0.0,1.0,2.0,1.0,1.0,1.5,0.0,15.0,34.0


In [4]:
dat_clin.columns

Index(['ID', 'Alter', 'Geschlecht', 'Krankheitsdauer', 'BDI-Score',
       'Schahmann semantische Wortflüssigkeit',
       'Schahmann phonematische Wortflüssigkeit',
       'Schahmann Kategorie-Wechsel', 'Schahmann Zahlen-spanne vorwärts',
       'Schahmann Zahlen-spanne rückwärts', 'Schahmann Würfel ',
       'Schahmann verbaler Abruf', 'Schahmann Gemeinsam-keiten',
       'Schahmann Go/No-Go', 'Schahmann Affekt',
       'Schahmann Gesamtwert Aufgaben', 'Schahmann Gesamtwert Module',
       'Tetras Sprechen', 'Tetras mit dem Löffel essen',
       'Tetras aus einem Glas trinken', 'Tetras Körper-hygiene',
       'Tetras Ankleiden', 'Tetras Schütten/Ein-schenken',
       'Tetras Tablett tragen', 'Tetras Schlüssel benutzen',
       'Tetras Schreiben', 'Tetras Arbeiten',
       'Tetras Tätigkeit mit schwerster Beeinträchti-gung',
       'Tetras Grad der Beeinträchti-gung ',
       'Tetras soziale Beeinträchti-gung', 'Tetras Gesamtscore Interview',
       'Tetras Kopftremor', 'Tetras Gesich

In [13]:
vars_oi = ['ID', 'Alter', 'Geschlecht', 'Krankheitsdauer', 'BDI-Score','Schahmann Gesamtwert Aufgaben','Schahmann Gesamtwert Module','Tetras Gesamtscore Aufgaben-skala','Gesamtscore Tetras']

dat_miriam = dat_clin[vars_oi]
dat_miriam['Gesamtscore Tetras'] = dat_miriam['Gesamtscore Tetras'].where(dat_miriam['Gesamtscore Tetras'] != 0, np.nan)
dat_miriam['group'] = np.where(dat_miriam['Tetras Gesamtscore Aufgaben-skala'].isna(),'hc','et')

In [14]:
dat_miriam

,ID,Alter,Geschlecht,Krankheitsdauer,BDI-Score,Schahmann Gesamtwert Aufgaben,Schahmann Gesamtwert Module,Tetras Gesamtscore Aufgaben-skala,Gesamtscore Tetras,group
0,p114,35.0,m,NaN,4.0,100.0,1.0,17.0,33.0,et
1,p115,46.0,w,NaN,2.0,0.0,0.0,17.5,35.5,et
2,p116,23.0,m,NaN,5.0,108.0,1.0,NaN,NaN,hc
3,p117,81.0,m,NaN,13.0,84.0,4.0,NaN,NaN,hc
4,p118,51.0,m,NaN,11.0,113.0,1.0,NaN,NaN,hc
5,p119,72.0,m,NaN,2.0,73.0,7.0,22.5,38.5,et
6,p120,81.0,m,NaN,2.0,73.0,4.0,18.5,39.5,et
7,p121,68.0,m,NaN,6.0,94.0,1.0,29.5,58.5,et
8,p214,NaN,NaN,NaN,NaN,103.0,1.0,NaN,NaN,hc
9,p109,52.0,m,NaN,4.0,107.0,0.0,15.0,34.0,et


In [36]:
import pandas as pd
import numpy as np


# Filter the dat_miriamframe to only include patients and controls
patients = dat_miriam[dat_miriam["group"] == "et"]
controls = dat_miriam[dat_miriam["group"] == "hc"]



# Convert the data into pandas dataframes
df1 = patients.copy()
df2 = controls.copy()

# Create a function to match individuals by gender and age
def match(group1, group2):
    matches = []
    matched_group2_indexes = []
    for i, row1 in group1.iterrows():
        min_difference = np.inf
        match_index = -1
        for j, row2 in group2.iterrows():
            if (row1['Geschlecht'] == row2['Geschlecht']) & (j not in matched_group2_indexes):
                difference = abs(row1['Alter'] - row2['Alter'])
                if difference < min_difference:
                    min_difference = difference
                    match_index = j
        if match_index != -1:
            matches.append((row1['ID'], group2.loc[match_index]['ID']))
            matched_group2_indexes.append(match_index)
    return matches

# Call the function to get the matches
matches = match(df1, df2)

# Print the matches
print(matches)



[('p114', 'p116'), ('p115', 'c306'), ('p119', 'c301'), ('p120', 'p117'), ('p121', 'c308'), ('p109', 'p118'), ('p112', 'p303'), ('p104', 'c307'), ('p202', 'c302'), ('p203', 'c309'), ('p204', 'c310'), ('p211', 'p113'), ('p212', 'c304')]
